In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
! pip install wandb

     |████████████████████████████████| 2.0MB 20.6MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 163kB 54.9MB/s 
     |████████████████████████████████| 133kB 59.0MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=c1c8bfe93e3d93edadc5608c04a7ac725c2efca837feea774061220ff1149c41
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=1fb2b02ae431d13db83865dcc20205b9a33e128b9642cd6118893bbb72578da0
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,BatchNormalization,Dropout,Activation ,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import CategoricalCrossentropy
import PIL
import wandb
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping
from keras import optimizers

In [4]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
def cnn(n,num_filters,ker_size_input,ker_size,activ,pool,num_nodes,filter_org,bn_vs_dp,bn=0,dp='no'):
  

  model = Sequential()
  model.add(Conv2D(num_filters, (ker_size_input, ker_size_input), input_shape=(224,224,3)))
  model.add(Activation(activ))
  model.add(MaxPooling2D(pool_size=(pool, pool)))

  for i in range(1,n):
  
    if filter_org=='same':
      num_filters= num_filters
    elif filter_org=='double':
      num_filters*=2
    else:
      num_filters//=2
      
        
    model.add(Conv2D(num_filters, (ker_size, ker_size)))

    if bn:
        model.add(BatchNormalization())
        
    model.add(Activation(activ))
    model.add(MaxPooling2D(pool_size=(pool, pool)))
      
  model.add(Flatten())
  model.add(Dense(num_nodes))
  
  if bn_vs_dp == 'dp':
    model.add(Dropout(dp))
  else:
    model.add(BatchNormalization())

  model.add(Activation(activ))
      
  model.add(Dense(10, activation='softmax'))

  return model




In [18]:
def train():
  default_hyperparams = dict(
    bn=0,
    num_filters=64,
    fliter_org='same',
    dropout=0,
    data_aug=0,
    learning_rate=0.01,
    epochs=5,
    activ="ReLU",
    optimizer = 'Adam',
    batch_size=32,
    ker_size_input=5,
    bn_vs_dp='bn'
  )

    
    
  wandb.init(config = default_hyperparams)
  config = wandb.config

  train_data_dir= '/content/gdrive/MyDrive/inaturalist_12K/train'

  batch_size=config.batch_size

  if config.data_aug:
      
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45, 
    width_shift_range=.15, 
    height_shift_range=.15, 
    horizontal_flip=True, 
    zoom_range=0.5,
    validation_split=0.1) 

  else:
      
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1) # set validation split
  
    

  train_it = train_datagen.flow_from_directory(
      train_data_dir,
      target_size=(224,224),
      batch_size=batch_size,
      class_mode='categorical',
      subset='training') # set as training data

  val_it = train_datagen.flow_from_directory(
      train_data_dir, # same directory as training data
      target_size=(224,224),
      batch_size=batch_size,
      class_mode='categorical',
      subset='validation') # set as validation data
  
  # Your model here ...
  model = cnn(5,config.num_filters,config.ker_size_input,3,config.activ,2,1024,config.filter_org,config.bn_vs_dp,config.bn,config.dropout)

  if config.optimizer == 'Adam':
    model.compile(optimizers.Adam(lr=config.learning_rate, decay=1e-6), loss=CategoricalCrossentropy(), metrics='acc')
  elif config.optimizer == 'rmsprop':
    model.compile(optimizers.RMSprop(lr=config.learning_rate, decay=1e-6), loss=CategoricalCrossentropy(), metrics='acc')
  else:
     model.compile(optimizers.SGD(lr=config.learning_rate, decay=1e-6), loss=CategoricalCrossentropy(), metrics='acc')
  
  model.fit(
  train_it,
  steps_per_epoch = train_it.samples //batch_size,
  validation_data = val_it, 
  validation_steps = val_it.samples // batch_size,
  epochs = config.epochs,
  callbacks=[WandbCallback(data_type='image',validation_data = val_it,verbose=1),EarlyStopping(patience=10,restore_best_weights=True)])
  #train_loss,train_accuracy = model.evaluate(train_it, callbacks=[WandbCallback()])
  #val_loss, val_accuracy = model.evaluate(val_it, callbacks=[WandbCallback()])
  #wandb.log({'val_loss':val_loss,'val_accuracy':val_accuracy*100,'train_loss': train_loss,'train_accuracy':train_accuracy*100 }) # wandb.log to track custom metrics

In [20]:

sweep_config = {
   #'program': train(),
    'method': 'bayes',         
    'metric': {
        'name': 'val_accuracy',     
        'goal': 'maximize'      
    },
    'parameters': {
        'learning_rate': {
            'values': [1e-3,1e-4]
        },
        'activ': {
            'values': ['relu']
        },
        'bn': {
            'values': [0,1]
        },
        'num_filters': {
            'values': [32,64]
        },
        'filter_org': {
            'values': ['double']
        },
        'epochs': {
            'values':[5]

        },
        'dropout': {
            'values': [0,0.2,0.3]
        },
        'data_aug': {
            'values': [0,1]
        },
        'optimizer' : {
            'values': ['Adam','rmsprop','sgd']
        },

        'batch_size': {
            'values': [32]
        },
        'ker_size_input':{
            'values':[5,7]
        },
        'bn_vs_dp':{
            'values':['bn,dp']
        }

        }
    }


sweep_id = wandb.sweep(sweep_config,project='asgn2 q1 1')

wandb.agent(sweep_id, function=train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: ncgbddq5
Sweep URL: https://wandb.ai/dl_assgn/asgn2%20q1%201/sweeps/ncgbddq5


wandb: Agent Starting Run: sniiglms with config:
wandb: 	activ: relu
wandb: 	batch_size: 32
wandb: 	bn: 1
wandb: 	bn_vs_dp: bn,dp
wandb: 	data_aug: 0
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	filter_org: double
wandb: 	ker_size_input: 7
wandb: 	learning_rate: 0.0001
wandb: 	num_filters: 64
wandb: 	optimizer: sgd
wandb: Ctrl + C detected. Stopping sweep.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [8]:
predict=model.predict(test_it, steps = len(test_it.filenames),verbose=1)

NameError: ignored

In [ ]:
predicted_class_indices=np.argmax(predict,axis=1)

In [ ]:
predicted_class_indices

In [ ]:
true=test_it.classes

In [ ]:
len(predictions)

In [ ]:
true=[]
for i in range(2000):

  a,b=test_it.next()
  true_class_indices=np.argmax(b,axis=1)
  true.extend(true_class_indices.tolist())
  

In [ ]:
c=0
for i in range(2000):
  if predicted_class_indices[i]==true[i]:
    c+=1

print(c/2000*100)


In [ ]:
import matplotlib.pyplot as plt
for i in range(2000):
  
  a,b = test_it.next()
  true_class_indices = np.argmax(b, axis=1)
  true_batch.append(a)
  true_label.append(true_class_indices)

  plt.imshow(a)
  plt.title(str(true_class_indices) + ',' + str(predicted_class_indices[i]))
